# Generate Clickstream Data
**This notebook generates clickstream data with explicit frequent viewing of products before placing an order.**

**THIS NOTEBOOK CAN BE RUN IN PARALLEL WITH `1_setup.ipynb`**

**Recommended settings to run this notebook in SageMaker Studio:**

- Image: Data Science
- Kernel: Python3
- Instance type: <font color='blue'>ml.m5.large (2 vCPU + 8 GiB)</font>

---

## Contents

1. [Background](#Background)

## Background


This is for tracking viewing habits and translating them to useful conversion figures.

## Setup 

### Prerequisites

In [1]:
!pip install Faker confluent-kafka

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 87.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 114.0 MB/s eta 0:00:00


#### Imports

In [2]:
from botocore.client import ClientError
from collections import defaultdict
from faker import Faker
import pandas as pd
import numpy as np
import sagemaker
import datetime
import hashlib
import random
import boto3
import math
import os

/opt/conda/lib/python3.11/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


#### Seed for Reproducibility

In [3]:
faker = Faker()
faker.seed_locale('en_US', 0)
SEED = 123
random.seed(SEED)
np.random.seed(SEED)
faker.seed_instance(SEED)

#### Constants

In [4]:
# TOTAL_UNIQUE_TRANSACTIONS = 5400000 # 5.4 Million
# TOTAL_UNIQUE_USERS = 10000
BUCKET = sagemaker.Session().default_bucket()

In [5]:
import json
import time
import random
from faker import Faker
from confluent_kafka import Producer

# Initialize Faker and Kafka Producer
fake = Faker()
# Possible event types
event_types = ['page_view', 'click', 'add_to_cart', 'purchase']

def delivery_report(err, msg):
    """Called once for each message produced to indicate delivery result."""
    if err is not None:
        print(f'Message delivery failed: {err}')
    else:
        print(f'Message delivered to {msg.topic()} [{msg.partition()}]')

def generate_dummy_event():
    """Generates a dummy customer interaction event."""
    return {
        'event_id': fake.uuid4(),
        'timestamp': fake.iso8601(),
        'customer_id': random.randint(1, 1000),
        'session_id': fake.uuid4(),
        'event_type': random.choice(event_types),
        'product_id': random.randint(1, 500),
        'product_category': random.choice(['electronics', 'fashion', 'home', 'books', 'toys']),
        'price': round(random.uniform(10.0, 500.0), 2)
    }

#### Simple generation of random events with no clear directions towards a purchase

In [7]:
import json
import time
import random
import datetime
from faker import Faker
from confluent_kafka import Producer

# Initialize Faker and Kafka Producer
fake = Faker()
# Define extended event types including scrolling behavior
event_types = ['page_view', 'scroll', 'click', 'add_to_cart', 'purchase']
def generate_session_events(customer_id):
    """Generates an ordered list of dummy events for a given session."""
    session_id = fake.uuid4()
    event_chain = []
    # Start with a random timestamp (e.g., within the current month)
    current_time = fake.date_time_this_month().strftime('%Y-%m-%d %H:%M:%S')
    # Random number of events for the session (between 5 and 10 events)
    n_events = random.randint(5, 10)
    
    for i in range(n_events):
        # Randomly choose an event type
        event_type = random.choice(event_types)
        # Increment time by a random delta (5 to 30 seconds) to simulate order
        current_dt = datetime.datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
        delta = datetime.timedelta(seconds=random.randint(5, 30))
        current_dt += delta
        current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
        
        event = {
            'event_id': fake.uuid4(),
            'timestamp': current_time,
            'customer_id': customer_id,
            'session_id': session_id,
            'event_type': event_type,
            'product_id': random.randint(1, 500),
            'product_category': random.choice(['electronics', 'fashion', 'home', 'books', 'toys']),
            'price': round(random.uniform(10.0, 500.0), 2),
            'order_in_session': i + 1  # Indicates the order of the event within the session
        }
        event_chain.append(event)
    return event_chain

# Continuously generate and send session events
currentSessions, totalSessions = 0, 10
allSessions = []
while currentSessions < totalSessions:
    # Pick a random customer
    customer_id = random.randint(1, 1000)
    session_events = generate_session_events(customer_id)
    allSessions.append(session_events)
    currentSessions += 1

In [103]:
print([len(sess) for sess in allSessions])
for event in allSessions[1][:]:
    print(event)
    print("---------------------------")

[6, 7, 10, 5, 9, 8, 6, 7, 10, 9]
{'event_id': '575ded00-8dd0-4300-a7e6-7818bb1150b5', 'timestamp': '2025-03-04 03:19:39', 'customer_id': 472, 'session_id': '6fe11191-4fea-42da-afc9-e2d5389adb88', 'event_type': 'purchase', 'product_id': 9, 'product_category': 'books', 'price': 265.23, 'order_in_session': 1}
---------------------------
{'event_id': '5e12d07f-de9d-48fd-b635-9e6124f6dd68', 'timestamp': '2025-03-04 03:19:55', 'customer_id': 472, 'session_id': '6fe11191-4fea-42da-afc9-e2d5389adb88', 'event_type': 'click', 'product_id': 110, 'product_category': 'books', 'price': 468.94, 'order_in_session': 2}
---------------------------
{'event_id': '388d26f5-a35a-4770-be1d-0b72ec7f03b9', 'timestamp': '2025-03-04 03:20:16', 'customer_id': 472, 'session_id': '6fe11191-4fea-42da-afc9-e2d5389adb88', 'event_type': 'page_view', 'product_id': 425, 'product_category': 'electronics', 'price': 300.48, 'order_in_session': 3}
---------------------------
{'event_id': '24b68581-3110-4c51-83f2-e5b8a03876c0

#### Generation of  events with clear directions towards a purchase

Based on these factors, users will indicate higher chances of an actual purchase:
- moderate dwell times
- 2 or more consecutive interactions of page clicks, views, scrolling

## Generating events without time restriction

In [37]:
import json
import time
import random
import datetime
from faker import Faker
from confluent_kafka import Producer

# Initialize Faker and Kafka Producer
fake = Faker()

# Define a dwell time distribution (in seconds) as (min, max, probability)
dwell_distribution = [
    (1, 3, 0.05),    # 10% of events: very quick interactions
    (3, 10, 0.70),   # 60% of events: typical dwell times
    (10, 30, 0.20),  # 20% of events: longer engagement
    (30, 60, 0.05)   # 10% of events: very long dwell times
]

def get_dwell_time():
    """Return a dwell time sampled from the defined distribution."""
    r = random.random()
    cumulative = 0
    for start, end, prob in dwell_distribution:
        cumulative += prob
        if r <= cumulative:
            return round(random.uniform(start, end), 2)
    return round(random.uniform(3, 10), 2)

def generate_session_events(customer_id):
    """
    Simulate a user session on a well-functioning site.
    
    - The session consists of one or more product chains.
    - Within each chain, a single product is focused on.
    - The user starts with a page_view and performs a series of events (page_view, scroll, click).
    - The chance to add to cart increases when:
        • There are multiple consecutive interactions (e.g. clicks),
        • And the dwell time between events is moderate.
    - At the end of the session, if any product was added to cart, a purchase event is generated.
    """
    session_id = fake.uuid4()
    events = []
    current_time = fake.date_time_this_month().strftime('%Y-%m-%d %H:%M:%S')
    order_in_session = 1
    cart = []  # Track product_ids added to cart

    # Decide the number of product chains in the session (e.g., 1 to 3)
    num_chains = random.randint(1, 10)
    
    for _ in range(num_chains):
        # For each chain, choose a single product to focus on
        product_id = random.randint(1, 500)
        product_category = random.choice(['electronics', 'fashion', 'home', 'books', 'toys'])
        consecutive_interactions = 0
        chain_length = random.randint(3, 6)
        
        for i in range(chain_length):
            dwell = get_dwell_time()
            # Update the timestamp based on the dwell time
            current_dt = datetime.datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
            current_dt += datetime.timedelta(seconds=dwell)
            current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # First event in chain is always a page_view
            if i == 0:
                event_type = 'page_view'
            else:
                # Choose randomly among browsing events
                event_type = random.choice(['page_view', 'scroll', 'click'])
                if event_type == 'click':
                    consecutive_interactions += 1
                else:
                    # Reduce consecutive interaction count slightly for non-clicks
                    consecutive_interactions = max(0, consecutive_interactions - 1)
            
            # Increase chance for add_to_cart if conditions are met
            if (i > 1 and product_id not in cart and consecutive_interactions >= 2 and 3 <= dwell <= 10):
                if random.random() < 0.7:
                    event_type = 'add_to_cart'
                    cart.append(product_id)
            
            event = {
                'event_id': fake.uuid4(),
                'timestamp': current_time,
                'customer_id': customer_id,
                'session_id': session_id,
                'event_type': event_type,
                'product_id': product_id,
                'product_category': product_category,
                'price': round(random.uniform(10.0, 500.0), 2),
                'order_in_session': order_in_session
            }
            events.append(event)
            order_in_session += 1

    # At the end of the session, if any products were added to the cart, generate a purchase event.
    if cart:
        dwell = get_dwell_time()
        current_dt = datetime.datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
        current_dt += datetime.timedelta(seconds=dwell)
        current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
        purchase_event = {
            'event_id': fake.uuid4(),
            'timestamp': current_time,
            'customer_id': customer_id,
            'session_id': session_id,
            'event_type': 'purchase',
            'purchased_items': cart,  # List of product_ids
            'total_amount': sum(round(random.uniform(10.0, 500.0), 2) for _ in cart),
            'order_in_session': order_in_session
        }
        events.append(purchase_event)
        order_in_session += 1

    # Optionally, the user may continue browsing after purchase (simulate one additional chain)
    if random.random() < 0.5:
        product_id = random.randint(1, 500)
        product_category = random.choice(['electronics', 'fashion', 'home', 'books', 'toys'])
        chain_length = random.randint(2, 4)
        for _ in range(chain_length):
            dwell = get_dwell_time()
            current_dt = datetime.datetime.strptime(current_time, '%Y-%m-%d %H:%M:%S')
            current_dt += datetime.timedelta(seconds=dwell)
            current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
            event = {
                'event_id': fake.uuid4(),
                'timestamp': current_time,
                'customer_id': customer_id,
                'session_id': session_id,
                'event_type': random.choice(['page_view', 'scroll']),
                'product_id': product_id,
                'product_category': product_category,
                'price': round(random.uniform(10.0, 500.0), 2),
                'order_in_session': order_in_session
            }
            events.append(event)
            order_in_session += 1

    return events

currentSessions, totalSessions = 0, 1
allSessions3 = []
# Continuously generate and send session events
while currentSessions < totalSessions:
    print(currentSessions)
    # Pick a random customer
    customer_id = random.randint(1, 1000)
    session_events = generate_session_events(customer_id)
    allSessions3.extend(session_events)
    currentSessions += 1

0


## Generating events WITH time restriction

In [6]:
import json
import time
import random
import datetime
from faker import Faker
from confluent_kafka import Producer

# Initialize Faker
fake = Faker()

# Define a dwell time distribution (in seconds) as (min, max, probability)
dwell_distribution = [
    (1, 3, 0.05),    # 5% of events: very quick interactions
    (3, 10, 0.70),   # 70% of events: typical dwell times
    (10, 30, 0.20),  # 20% of events: longer engagement
    (30, 60, 0.05)   # 5% of events: very long dwell times
]

def get_dwell_time():
    """Return a dwell time sampled from the defined distribution."""
    r = random.random()
    cumulative = 0
    for start, end, prob in dwell_distribution:
        cumulative += prob
        if r <= cumulative:
            return round(random.uniform(start, end), 2)
    return round(random.uniform(3, 10), 2)

def generate_session_events_period(customer_id, start_time, end_time):
    """
    Simulate a user session that generates events only between start_time and end_time.
    
    Parameters:
        customer_id (int): The customer identifier.
        start_time (str): Session start time in '%Y-%m-%d %H:%M:%S' format.
        end_time (str): Session end time in '%Y-%m-%d %H:%M:%S' format.
    
    Returns:
        list: A list of event dictionaries.
    """
    # Convert string inputs to datetime objects
    current_dt = datetime.datetime.strptime(start_time, '%Y-%m-%d %H:%M:%S')
    end_dt = datetime.datetime.strptime(end_time, '%Y-%m-%d %H:%M:%S')
    
    session_id = fake.uuid4()
    events = []
    order_in_session = 1
    cart = []  # Track product_ids added to cart

    # Decide the number of product chains in the session
    num_chains = random.randint(1, 10)
    
    for _ in range(num_chains):
        # Check if there's still time for another chain
        if current_dt >= end_dt:
            break

        product_id = random.randint(1, 500)
        product_category = random.choice(['electronics', 'fashion', 'home', 'books', 'toys'])
        consecutive_interactions = 0
        chain_length = random.randint(3, 6)
        
        for i in range(chain_length):
            # Check if adding a new event would exceed end time
            dwell = get_dwell_time()
            next_dt = current_dt + datetime.timedelta(seconds=dwell)
            if next_dt > end_dt:
                # Stop generating further events if we've reached the end
                return events
            current_dt = next_dt
            
            current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
            
            # Determine event type
            if i == 0:
                event_type = 'page_view'
            else:
                event_type = random.choice(['page_view', 'scroll', 'click'])
                if event_type == 'click':
                    consecutive_interactions += 1
                else:
                    consecutive_interactions = max(0, consecutive_interactions - 1)
            
            # Increase chance for add_to_cart if conditions are met
            if (i > 1 and product_id not in cart and consecutive_interactions >= 2 and 3 <= dwell <= 10):
                if random.random() < 0.7:
                    event_type = 'add_to_cart'
                    cart.append(product_id)
            
            event = {
                'event_id': fake.uuid4(),
                'timestamp': current_time,
                'customer_id': customer_id,
                'session_id': session_id,
                'event_type': event_type,
                'product_id': product_id,
                'product_category': product_category,
                'price': round(random.uniform(10.0, 500.0), 2),
                'order_in_session': order_in_session
            }
            events.append(event)
            order_in_session += 1
        
    # If any product was added to cart, generate a purchase event before end_time
    if cart and current_dt < end_dt:
        dwell = get_dwell_time()
        next_dt = current_dt + datetime.timedelta(seconds=dwell)
        if next_dt <= end_dt:
            current_dt = next_dt
            current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
            purchase_event = {
                'event_id': fake.uuid4(),
                'timestamp': current_time,
                'customer_id': customer_id,
                'session_id': session_id,
                'event_type': 'purchase',
                'purchased_items': cart,
                'total_amount': sum(round(random.uniform(10.0, 500.0), 2) for _ in cart),
                'order_in_session': order_in_session
            }
            events.append(purchase_event)
            order_in_session += 1

    # Optionally, simulate additional browsing if time permits
    if random.random() < 0.5 and current_dt < end_dt:
        product_id = random.randint(1, 500)
        product_category = random.choice(['electronics', 'fashion', 'home', 'books', 'toys'])
        chain_length = random.randint(2, 4)
        for _ in range(chain_length):
            dwell = get_dwell_time()
            next_dt = current_dt + datetime.timedelta(seconds=dwell)
            if next_dt > end_dt:
                break
            current_dt = next_dt
            current_time = current_dt.strftime('%Y-%m-%d %H:%M:%S')
            event = {
                'event_id': fake.uuid4(),
                'timestamp': current_time,
                'customer_id': customer_id,
                'session_id': session_id,
                'event_type': random.choice(['page_view', 'scroll']),
                'product_id': product_id,
                'product_category': product_category,
                'price': round(random.uniform(10.0, 500.0), 2),
                'order_in_session': order_in_session
            }
            events.append(event)
            order_in_session += 1

    return events

# Example usage:
start_time = "2025-03-01 02:01:45"
end_time = "2025-03-08 14:33:55"
customer_id = random.randint(1, 1000)
session_events = generate_session_events_period(customer_id, start_time, end_time)
# print(json.dumps(session_events, indent=2))


currentSessions, totalSessions = 0, 10
allSessions3 = []
# Continuously generate and send session events
while currentSessions < totalSessions:
    print(currentSessions)
    # Pick a random customer
    customer_id = random.randint(1, 1000)
    session_events = generate_session_events_period(customer_id, start_time, end_time)
    allSessions3.extend(session_events)
    currentSessions += 1


0
1
2
3
4
5
6
7
8
9


In [7]:
df_all = pd.DataFrame(allSessions3)
print(len(df_all))
df_all.head()

274


,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount
0,9268038f-dbef-4e0c-80e2-a5d7866c6000,2025-03-01 02:01:52,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,page_view,453.0,books,192.33,1,NaN,NaN
1,bf953c32-9485-46e3-8fba-229a57e6e588,2025-03-01 02:01:56,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,click,453.0,books,38.01,2,NaN,NaN
2,e6bd47c5-1485-45b6-b347-56ddda312547,2025-03-01 02:02:03,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,page_view,453.0,books,457.72,3,NaN,NaN
3,38dd0f35-4b17-4255-99a8-b1370e7fd726,2025-03-01 02:02:09,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,page_view,396.0,home,175.48,4,NaN,NaN
4,04f79f4e-0bbc-4c1d-a6e5-276162623e98,2025-03-01 02:02:13,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,click,396.0,home,199.26,5,NaN,NaN


In [8]:
int_vals = {
    "purchase": 0,
    "page_view": 4,
    "scroll": 2,
    "click": 1,
    "add_to_cart": 6
}
df_all["interaction_value"] = df_all["event_type"].map(int_vals)

In [9]:
print(len(df_all[df_all["event_type"] == "purchase"]))
df_all[df_all["event_type"] == "click"]

6


,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount,interaction_value
1,bf953c32-9485-46e3-8fba-229a57e6e588,2025-03-01 02:01:56,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,click,453.0,books,38.01,2,NaN,NaN,1
4,04f79f4e-0bbc-4c1d-a6e5-276162623e98,2025-03-01 02:02:13,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,click,396.0,home,199.26,5,NaN,NaN,1
6,a4f115ef-5208-42b9-88f4-c224bf695dce,2025-03-01 02:02:25,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,click,396.0,home,337.73,7,NaN,NaN,1
8,1e0c98f8-9369-4909-ac94-5bc6008f4bd2,2025-03-01 02:02:41,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,click,396.0,home,77.31,9,NaN,NaN,1
19,78ff196d-f901-439f-bb12-5e4580c296bc,2025-03-01 02:02:35,139,099b0f4b-8831-42b5-9091-c7be2018f892,click,405.0,toys,429.34,5,NaN,NaN,1
23,fbbfc93b-66dd-4857-84ab-1b749e7be780,2025-03-01 02:03:46,139,099b0f4b-8831-42b5-9091-c7be2018f892,click,405.0,toys,194.44,9,NaN,NaN,1
25,f871a5e0-7f76-4d91-af07-a12601a3f457,2025-03-01 02:04:12,139,099b0f4b-8831-42b5-9091-c7be2018f892,click,36.0,home,200.32,11,NaN,NaN,1
29,7a011cb6-2d9f-49fd-8785-5ab7bc010d65,2025-03-01 02:04:44,139,099b0f4b-8831-42b5-9091-c7be2018f892,click,137.0,books,74.63,15,NaN,NaN,1
36,808244e3-a8e9-40ff-9aac-e94d46f18f3d,2025-03-01 02:05:39,139,099b0f4b-8831-42b5-9091-c7be2018f892,click,461.0,toys,464.18,22,NaN,NaN,1
38,869cf6c1-9f5d-4746-a4fa-ae71a42d080c,2025-03-01 02:05:52,139,099b0f4b-8831-42b5-9091-c7be2018f892,click,430.0,fashion,277.03,24,NaN,NaN,1


In [10]:
df_grouped = df_all.groupby(["customer_id"])
df_grouped.agg({'event_type': ','.join}) 

,event_type
customer_id,
25,"page_view,click,page_view,page_view,click,add_..."
118,"page_view,scroll,page_view,page_view,scroll,sc..."
130,"page_view,page_view,scroll,scroll,page_view,sc..."
139,"page_view,page_view,page_view,page_view,click,..."
176,"page_view,scroll,click,scroll,click,click,page..."
269,"page_view,scroll,scroll,scroll,page_view,page_..."
448,"page_view,click,page_view,page_view,scroll,cli..."
660,"page_view,click,page_view,click,add_to_cart,pa..."
665,"page_view,page_view,scroll,click,page_view,cli..."


In [11]:
df_all['cumsum_interactions'] = df_all.groupby(['customer_id'])['interaction_value'].cumsum()

#view updated DataFrame
df_all

,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount,interaction_value,cumsum_interactions
0,9268038f-dbef-4e0c-80e2-a5d7866c6000,2025-03-01 02:01:52,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,page_view,453.0,books,192.33,1,NaN,NaN,4,4
1,bf953c32-9485-46e3-8fba-229a57e6e588,2025-03-01 02:01:56,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,click,453.0,books,38.01,2,NaN,NaN,1,5
2,e6bd47c5-1485-45b6-b347-56ddda312547,2025-03-01 02:02:03,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,page_view,453.0,books,457.72,3,NaN,NaN,4,9
3,38dd0f35-4b17-4255-99a8-b1370e7fd726,2025-03-01 02:02:09,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,page_view,396.0,home,175.48,4,NaN,NaN,4,13
4,04f79f4e-0bbc-4c1d-a6e5-276162623e98,2025-03-01 02:02:13,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,click,396.0,home,199.26,5,NaN,NaN,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...
269,bc5d6f55-625c-4b34-ad74-338a80bdeac4,2025-03-01 02:03:15,269,56fb9d68-9990-4fcf-bed7-ba808ea63bef,page_view,438.0,electronics,81.18,11,NaN,NaN,4,31
270,160acce8-6dcc-41f9-900a-3cc1e26d1c25,2025-03-01 02:04:02,269,56fb9d68-9990-4fcf-bed7-ba808ea63bef,click,438.0,electronics,459.37,12,NaN,NaN,1,32
271,997a8254-b620-4bae-874f-42ff0eb77279,2025-03-01 02:04:06,269,56fb9d68-9990-4fcf-bed7-ba808ea63bef,page_view,429.0,toys,237.79,13,NaN,NaN,4,36
272,533678e7-192f-401e-ac47-909b69ccae1e,2025-03-01 02:04:14,269,56fb9d68-9990-4fcf-bed7-ba808ea63bef,click,429.0,toys,260.78,14,NaN,NaN,1,37


In [12]:
print("No of customers who made a purchase:", len(df_all[df_all["event_type"] == "purchase"]))
df_all[df_all["event_type"] == "purchase"]

No of customers who made a purchase: 6


,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount,interaction_value,cumsum_interactions
14,e70dc13d-80d1-41d7-b9e0-97e3289c3068,2025-03-01 02:03:40,25,0816bcd6-6298-4ab1-8cd8-a2a30bd4de10,purchase,NaN,NaN,NaN,15,[396],446.35,0,42
51,629cb977-085e-4c21-a9a4-b6c59435c13c,2025-03-01 02:07:17,139,099b0f4b-8831-42b5-9091-c7be2018f892,purchase,NaN,NaN,NaN,37,"[36, 137]",425.38,0,108
110,d5ff9b84-79a6-48e1-86db-a196be42f386,2025-03-01 02:03:43,660,939478d1-4284-419f-84e9-3b2eb8ec2340,purchase,NaN,NaN,NaN,12,[401],384.13,0,33
174,3d3679a2-5c08-42a8-b388-7038b4e525a6,2025-03-01 02:10:26,448,262a60ef-eb3d-4a4d-a7e1-816b91f16784,purchase,NaN,NaN,NaN,41,[196],376.75,0,116
202,542e3238-3224-47c9-b7a8-36a5e6b8981a,2025-03-01 02:05:03,665,24f56da2-81bd-477a-a70f-77980e2b1baf,purchase,NaN,NaN,NaN,24,[244],123.99,0,67
227,7b9d2ace-01ff-4b02-a73f-7710d0a75b22,2025-03-01 02:04:59,176,f682e7cf-bb1e-44ed-a0e6-95c445494aa6,purchase,NaN,NaN,NaN,23,[109],493.29,0,62


In [13]:
data_dir = os.path.join(os.getcwd(), 'data/raw_clicks')
os.makedirs(data_dir, exist_ok=True)

In [14]:
# df_all.to_csv("data/raw_clicks/clickstream.csv", index=False)
df_all.to_csv(f's3://{BUCKET}/raw_clicks/clickstream.csv', index=False)

In [15]:
df_all = pd.read_csv(f's3://{BUCKET}/raw_clicks/clickstream.csv')
df_all["timestamp"] = pd.to_datetime(df_all["timestamp"])
print(type(df_all["timestamp"].head()[0]))
print(df_all["timestamp"].min())
print(df_all["timestamp"].max())

<class 'pandas._libs.tslibs.timestamps.Timestamp'>
2025-03-01 02:01:49
2025-03-01 02:10:51


In [16]:
TOTALLENGTH = len(df_all)
print(f"TOTALLENGTH: {TOTALLENGTH}")
# df_all = df_all[:TOTALLENGTH//4]

TOTALLENGTH: 274


In [17]:
# len(df_all)
df_all.describe()

,timestamp,customer_id,product_id,price,order_in_session,total_amount,interaction_value,cumsum_interactions
count,274,274.000000,268.000000,268.000000,274.000000,6.000000,274.000000,274.000000
mean,2025-03-01 02:04:32.894160896,317.985401,260.779851,266.506381,16.536496,374.981667,2.784672,47.182482
min,2025-03-01 02:01:49,25.000000,4.000000,10.170000,1.000000,123.990000,0.000000,4.000000
25%,2025-03-01 02:02:45.500000,130.000000,135.000000,140.672500,7.000000,378.595000,2.000000,21.000000
50%,2025-03-01 02:04:05,176.000000,257.000000,260.005000,14.000000,404.755000,2.000000,40.000000
75%,2025-03-01 02:05:51.750000128,448.000000,402.000000,399.220000,24.000000,441.107500,4.000000,69.000000
max,2025-03-01 02:10:51,827.000000,475.000000,499.830000,45.000000,493.290000,6.000000,126.000000
std,NaN,246.187724,146.985235,141.341637,11.340571,130.164456,1.385820,32.549364


# Generating Test Events after the train events for feature generation and prediction

In [18]:
df_all = pd.read_csv(f's3://{BUCKET}/raw_clicks/clickstream.csv')
df_all["timestamp"] = pd.to_datetime(df_all["timestamp"])
customerSet = df_all["customer_id"].unique()
print(customerSet)

[ 25 139 130 660 827 448 665 176 118 269]


In [19]:

# Example usage:
start_time = "2025-03-08 14:33:56"
end_time = "2025-03-12 02:01:45"
customer_id = random.randint(1, 1000)
# session_events = generate_session_events_period(customer_id, start_time, end_time)
# print(json.dumps(session_events, indent=2))


currentSessions, totalSessions = 0, 10
allSessions3 = []
# Continuously generate and send session events
while currentSessions < totalSessions:
    print(currentSessions)
    # Pick a random customer
    # customer_id = random.randint(1, 1000)
    # can only be from 
    session_events = generate_session_events_period(customer_id, start_time, end_time)
    allSessions3.extend(session_events)
    currentSessions += 1


0
1
2
3
4
5
6
7
8
9


In [24]:
df_test = pd.DataFrame(allSessions3)
print(len(df_test))
df_test.head()

228


,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount
0,adcf164e-4994-47e0-b9ac-2b020a237052,2025-03-08 14:34:03,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,page_view,109.0,books,272.57,1,NaN,NaN
1,d41bee05-95db-48e5-87ec-9e4d709cde8b,2025-03-08 14:34:12,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,page_view,109.0,books,254.77,2,NaN,NaN
2,9394a942-9391-4391-aaf7-b21c6ec9e444,2025-03-08 14:34:16,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,click,109.0,books,253.55,3,NaN,NaN
3,5c33e011-ad51-457d-93c2-defc159b6516,2025-03-08 14:35:02,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,click,109.0,books,148.57,4,NaN,NaN
4,26f32593-f4ca-4d85-86f9-c8204ca84aad,2025-03-08 14:35:06,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,page_view,109.0,books,403.23,5,NaN,NaN


In [21]:
int_vals = {
    "purchase": 0,
    "page_view": 4,
    "scroll": 2,
    "click": 1,
    "add_to_cart": 6
}
df_test["interaction_value"] = df_test["event_type"].map(int_vals)
df_test['cumsum_interactions'] = df_test.groupby(['customer_id'])['interaction_value'].cumsum()

In [22]:
print("No of customers who made a purchase:", len(df_test[df_test["event_type"] == "purchase"]))
df_test[df_test["event_type"] == "purchase"]

No of customers who made a purchase: 5


,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount,interaction_value,cumsum_interactions
25,23869c5f-6125-4169-9f2f-cc4d8dba0045,2025-03-08 14:39:14,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,purchase,NaN,NaN,NaN,26,[484],149.14,0,65
61,a30c372e-0f13-4e65-a9d4-d82cc907a1e2,2025-03-08 14:40:23,620,3214c8f5-018e-4c27-861d-d1a945a8fbc5,purchase,NaN,NaN,NaN,36,[445],148.39,0,167
92,2b66fc48-9691-4041-bcb1-e796d2f3e11c,2025-03-08 14:38:38,620,5ff70f74-bc30-410f-8af5-910e56390255,purchase,NaN,NaN,NaN,31,[428],399.19,0,249
151,efe41cb6-4dc9-4820-89c1-6bfff765f551,2025-03-08 14:42:58,620,b3643887-1661-4b34-b875-7e39ff6d41e9,purchase,NaN,NaN,NaN,51,[454],61.20,0,410
218,895b32dc-022d-4b40-8a18-dc54b47a74ee,2025-03-08 14:37:08,620,19415f64-4f54-494f-900b-f3cb3a887b37,purchase,NaN,NaN,NaN,16,"[279, 262]",403.85,0,587


In [25]:
df_test.to_csv(f's3://{BUCKET}/raw_clicks/clickstream_test.csv', index=False)

## reloading test data

In [27]:
df_test = pd.read_csv(f's3://{BUCKET}/raw_clicks/clickstream_test.csv')
df_test["timestamp"] = pd.to_datetime(df_test["timestamp"])
df_test.head()

,event_id,timestamp,customer_id,session_id,event_type,product_id,product_category,price,order_in_session,purchased_items,total_amount
0,adcf164e-4994-47e0-b9ac-2b020a237052,2025-03-08 14:34:03,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,page_view,109.0,books,272.57,1,NaN,NaN
1,d41bee05-95db-48e5-87ec-9e4d709cde8b,2025-03-08 14:34:12,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,page_view,109.0,books,254.77,2,NaN,NaN
2,9394a942-9391-4391-aaf7-b21c6ec9e444,2025-03-08 14:34:16,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,click,109.0,books,253.55,3,NaN,NaN
3,5c33e011-ad51-457d-93c2-defc159b6516,2025-03-08 14:35:02,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,click,109.0,books,148.57,4,NaN,NaN
4,26f32593-f4ca-4d85-86f9-c8204ca84aad,2025-03-08 14:35:06,620,98a20a7f-fb26-4862-9014-f9ccce54e7bf,page_view,109.0,books,403.23,5,NaN,NaN
